In [48]:
# merging datasets
import pandas as pd
ptv = pd.read_parquet('../data/curated/final_train_hospital_cbd_dist_data')
shop = pd.read_csv('../data/curated/agg_shopping_centre.csv')
household = pd.read_csv('../data/curated/dwellings_household.csv')
recreation = pd.read_csv('../data/curated/recreation_cleaned.csv')
population = pd.read_csv('../data/curated/total_population.csv')
park_domain = pd.read_parquet('../data/curated/parkres/parkres_domain.parquet')


In [50]:
ptv.head()

,url,price,address,property_type,latitude,longitude,Beds,Baths,Parking,bond,...,suburb,property_id,nearest_station_id,train_station_distance_km,cbd_distance_km,nearest_hospital_id,nearest_hospital_distance,log_nearest_station_distance,log_nearest_hospital_distance,log_cbd_distance
0,https://www.domain.com.au/10-allara-court-donv...,"$1,400.00","10 Allara Court, Donvale VIC 3111",Townhouse,-37.774273,145.181126,4.0,3.0,2.0,9125.0,...,Donvale,0,19900,5.6596,25.93487,H5063,1.920280,1.896059,1.071680,3.293422
1,https://www.domain.com.au/10-51-55-leslie-stre...,$500 Per Week,"10/51-55 Leslie Street, Donvale VIC 3111",Apartment / Unit / Flat,-37.781431,145.181474,2.0,1.0,1.0,2173.0,...,Donvale,7,19900,4.6923,24.88939,H5063,1.272015,1.739114,0.820667,3.253833
2,https://www.domain.com.au/293-richardson-stree...,$630.00,"293 Richardson Street, Carlton North VIC 3054",House,-37.784081,144.965615,2.0,1.0,0.0,2738.0,...,Carlton North,19,19969,1.4377,4.14098,H0372,1.651170,0.891055,0.975001,1.637244
3,https://www.domain.com.au/786-drummond-street-...,$720.00,"786 Drummond Street, Carlton North VIC 3054",House,-37.786146,144.971258,3.0,1.0,0.0,3129.0,...,Carlton North,22,19969,2.2956,4.03091,H0381,1.577533,1.192588,0.946833,1.615601
4,https://www.domain.com.au/8-1068-lygon-street-...,$750 pw,"8/1068 Lygon Street, Carlton North VIC 3054",Apartment / Unit / Flat,-37.780942,144.970534,3.0,2.0,2.0,3260.0,...,Carlton North,26,19969,1.7662,4.22506,H0372,1.990258,1.017475,1.095360,1.653466


In [51]:
joint = pd.merge(ptv, shop, on='sa2_code', how='left')
joint = joint.merge(household, left_on='sa2_code', right_on='SA2 code', how='left')
joint = joint.merge(recreation, on='sa2_code', how='left')
joint = joint.merge(population, on='sa2_code', how='left')
joint = joint.merge(park_domain, on='url', how='left')



In [41]:
joint = joint.drop(columns=['sa2_name_y', 'sa2_name_x'])

In [52]:
joint.columns

Index(['url', 'price', 'address_x', 'property_type', 'latitude', 'longitude',
       'Beds', 'Baths', 'Parking', 'bond', 'extracted_price_x', 'geometry',
       'index_right', 'sa2_code_x', 'sa2_name_x', 'chg_flag', 'chg_lbl',
       'sa3_code', 'sa3_name', 'sa4_code', 'sa4_name', 'gcc_code', 'gcc_name',
       'ste_code', 'ste_name', 'aus_code', 'aus_name', 'areasqkm', 'loci_uri',
       'geometry_proj', 'suburb', 'property_id', 'nearest_station_id',
       'train_station_distance_km', 'cbd_distance_km', 'nearest_hospital_id',
       'nearest_hospital_distance', 'log_nearest_station_distance',
       'log_nearest_hospital_distance', 'log_cbd_distance', 'mean_stores',
       'total_stores', 'count_shopping_centres', 'SA2 code', 'Region',
       'ERP_2021', 'ERP_2026', 'ERP_2031', 'ERP_2036', 'PNPD_2021',
       'PNPD_2026', 'PNPD_2031', 'PNPD_2036', 'POPD_2021', 'POPD_2026',
       'POPD_2031', 'POPD_2036', 'OPD_2021', 'OPD_2026', 'OPD_2031',
       'OPD_2036', 'HHS_2021', 'HHS_2026', 

In [53]:
print(len(joint))

9565
